In [2]:
import collections
import math
import os
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf


In [3]:
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)


def read_data(filename):
    ''' zip 파일에 포함된 텍스트 파일을 읽어서 단어 리스트 생성. 포함된 파일은 1개. 
    zip 파일은 30mb, txt 파일은 100mb. '''
    with zipfile.ZipFile(filename) as f:
        names = f.namelist()                # ['text8']
        contents = f.read(names[0])         # 크기 : 100,000,000바이트
        text = tf.compat.as_str(contents)   # 크기 : 100,000,000
        return text.split()                 # 갯수 : 17005207


vocabulary = read_data(filename)
print('Data size', len(vocabulary))   

# .을 기준으로 리스트 여러개로 나눈후에 space로 다시 리스트로 나눈다.
# I like you. you like me -> [[I like you], [you like me]] -> [[[I] [like] [you]], ... ]

Found and verified text8.zip
Data size 17005207


In [53]:
def word_numbering(vocabulary, number_of_n_words=50000):
    """
    Arguments:
    vocabulary -- a list of words you want to train the word2vec
    number_of_n_words -- number of most frequent n words in the vocabulary you want to set
    
    Returns:
    int_voc -- a list of vocabulary that is mapped into integer-valeud index
    word_to_int -- python dict mapping words in the vocabulary into an integer-valued index
    int_to_word -- python dict mapping integer-valued index to words
    most_frequent_n_words -- a list with pairs of n most frequent words with its frequency -> [(word, frequency)]
    """
    word_count = collections.Counter(vocabulary)
    most_frequent_n_words = word_count.most_common(number_of_n_words - 1)
            
    word_to_int = {'UNK': 0} # set 0 as unknown token
    for word, _ in most_frequent_n_words:
        word_to_int[word] = len(word_to_int)
    
    int_to_word = {v: k for k, v in word_to_int.items()} # reverse dict of word_to_int


    count = 0
    int_voc = []

    for word in vocabulary:
        if word in word_to_int: # if word is in n most frequent words
            int_voc.append(word_to_int[word]) #. change the word into the corresponding integer
        else:
            int_voc.append(0) # not in the n most frequent word.
            count += 1   # number of words that are not in n most frequent words
 
    most_frequent_n_words.insert(0, ('UNK', count))
    #most_frequent_n_words = np.insert(most_frequent_n_words, 0,('UNK', count),axis=0)
            
    return int_voc, word_to_int, int_to_word, most_frequent_n_words
    
# del vocabulary

In [54]:
# For the test
int_voc, word_to_int, int_to_word, most_frequent_n_words = word_numbering(vocabulary)

In [ ]:
def compute_f(most_frequent_n_words):
    """
    Arguments:
    
    Returns:
    """
    total = sum([most_frequent_n_words[x][1] for x in range(len(most_frequent_n_words))]) # compute the vocabulary len.
    
    
    return f

In [ ]:
def subsampling(int_voc, most_frequent_n_words, word, t=0.00001):
    """
    Arguments:
    most_frequent_n_words --
    t --
    
    Returns:
    p_w_i
    """
    
    

In [55]:
# For the test
total = sum([most_frequent_n_words[x][1] for x in range(len(most_frequent_n_words))])
total

17005207

In [21]:
a = [(1,2),(2,1),(3,1)]
b = np.array(a)
b[2,:]

array([3, 1])

In [42]:

most_frequent_n_words[1][1].astype(np.float)


1061396.0

In [ ]:
def create_mini_batch(int_voc, batch_size, window_size, num_context_word, start_x, start_y):
    """
    Create mini batch size of batch_size with each batch containing num_target_word training examples.
    
    Arguments:
    data -- a list of vocabulary that is mapped into integer-valeud index
    batch_size -- size of batch in stochastic gradient descent
    window_size -- size of window around the target word.
    num_context_word --
    
    Returns:
    start_x
    start_y
    """
    int_voc = [int_voc] # this will deleted if the input includes punctuation
    
    assert (start_y < len(int_voc[start_x]))

    for i in range(batch_size / num_context_word):
        start = max(0, start_y - window_size) 
        end = min(start_y + window_size, len(int_voc[start_x]) - 1)
    
    

In [ ]:
# Step 3: skip-gram 모델에 사용할 학습 데이터를 생성할 함수 작성

# 원본에서는 data_index를 전역변수로 처리했는데, 여기서는 매개변수와 반환값으로 변경했다.
# data 변수 또한 전역변수였는데, 첫 번째 매개변수로 전달하도록 변경했다.
def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    ''' Stochastic Gradient Descent 알고리즘에 사용할 minibatch 생성.
    :param data : 단어 인덱스 리스트
    :param batch_size : SGD 알고리즘에 적용할 데이터 갯수. 한 번에 처리할 크기.
    :param num_skips : context window에서 구축할 (target, context) 쌍의 갯수.
    :param skip_window : skip-gram 모델에 사용할 윈도우 크기.
         1이라면 목표 단어(target) 양쪽에 1개 단어이므로 context window 크기는 3이 된다. (단어, target, 단어)
         2라면 5가 된다. (단어 단어 target 단어 단어)
    :param data_index : 첫 번째 context window에 들어갈 data에서의 시작 위치.
    '''
    # 조건이 False라면 비정상 종료
    # num_skips <= batch_size. batch_size는 num_skips의 정수 배.
    # num_skips는 skip_window의 2배 이하.
    # num_skips가 skip_window의 2배일 때, context 윈도우의 모든 쌍 구성
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    temp = 'batch_size {}, num_skips {}, skip_window {}, data_index {}'
    # 최초 : batch_size 8, num_skips 2, skip_window 1, data_index 0
    # 학습 : batch_size 128, num_skips 2, skip_window 1, data_index 640000
    #       data_index는 64로 시작해서 64씩 증가한다. 나머지는 변경되지 않는다.
    # print(temp.format(batch_size, num_skips, skip_window, data_index))

    # ndarray에 값을 주지 않았다면 난수가 사용된다. 앞부분 10개만 표시.
    # batch는 1차원, labels는 2차원.
    # batch  : [0 0 268907754 536870912 -1354956798 32767 32229680 131073]
    # labels : [[0] [0] [268799296] [-805306368] [2] [0] [268811838] [131072]]
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    # span은 assert에 기술한 코드 때문에 항상 num_skips보다 크다.
    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    # deque
    # 처음과 마지막의 양쪽 끝에서 일어나는 입출력에 대해 가장 좋은 성능을 내는 자료구조
    # maxlen 옵션이 없으면 크기 제한도 없고, 있다면 지정한 크기만큼만 사용 가능.
    # maxlen을 3으로 전달하면 3개만 저장할 수 있고, 새로운 요소를 추가하면 반대쪽 요소가 자동으로 삭제됨.
    # 여기서는 자동 삭제 기능 때문에 사용.
    # data_index 번째부터 span 크기만큼 단어 인덱스 저장
    # 첫 번째 context 윈도우 구성
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    # 위의 코드를 재구성(에러).
    # data_index는 마지막에서 처음으로 cycle을 구성하기 때문에 오류 발생할 수 있다.
    # buffer = collections.deque(data[data_index:data_index+span], maxlen=span)
    # data_index = (data_index + span) % len(data)

    # skip-gram은 타겟 단어로부터 주변의 컨텍스트 단어를 예측하는 모델이다.
    # 학습하기 전에 단어들을 (target, context) 형태로 변환해 주어야 한다.
    # 바깥 루프는 batch_size // num_skips
    # 안쪽 루프는 num_skips
    # batch_size는 num_skips로 나누어 떨어지기 때문에 정확하게 batch_size만큼 반복
    for i in range(batch_size // num_skips):

        # 원본 코드
        # target = skip_window               # target label은 buffer의 가운데 위치
        # targets_to_avoid = [skip_window]
        #
        # for j in range(num_skips):
        #     while target in targets_to_avoid:
        #         target = random.randrange(span)
        #     targets_to_avoid.append(target)
        #     batch[i * num_skips + j] = buffer[skip_window]
        #     labels[i * num_skips + j, 0] = buffer[target]
        #     print(target, '**')       # (2, 0), (0, 2), (0, 2), (0, 2)

        # 재구성한 코드
        # skip_window는 context의 가운데 위치한 단어의 인덱스
        # skip_window가 3이라면 주변에 3개의 단어씩 위치하게 되고 3+1+3은 7개로 구성된 context가 만들어진다.
        # 읽어올 데이터가 0부터 시작한다면 skip_window는 3이 되고, context의 가운데에 위치한다.
        # 값을 생성할 때, num_skips와 span 중에서 신중하게 선택해야 한다.
        # num_skips는 context로부터 구성할 단어들의 갯수이기 때문에
        # num_skips를 사용하면 context에 포함된 단어들의 모든 인덱스가 반영되지 않을 수 있다.
        # skip_window*2 == num_skips 일 때는 모든 단어를 사용하기 때문에 난수를 사용할 필요가 없다.
        # 여기서는 항상 skip_window는 1, num_skips는 2의 값을 갖는다.
        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        # batch : target 단어만 들어가고, num_skips만큼 같은 단어가 중복된다.
        # labels : target을 제외한 단어만 들어가고, num_skips만큼 중복될 수 있다.
        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        # span이 큰 값이기 때문에 targets에 포함된 모든 값을 사용하지 않을 수 있다.
        # buffer는 numpy 데이터가 아니라서 슬라이스 연산 불가. 어쩔 수 없이 반복문 사용.
        # numpy 배열을 사용하면서 마지막에 있는 num_skips 갯수만큼 사용하는 것이 나을 수도 있다.
        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]
            # print(targets[j], '**')     # (2, 0), (0, 2), (0, 2), (0, 2)

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    # 아래는 generate_batch 함수를 최초 호출한 결과.
    # 오른쪽 출력 결과를 보면, batch와 labels의 관계를 알 수 있다.
    # print(buffer)           # deque([195, 2, 3134], maxlen=3)
    # print(data[:20])        # [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]
    # print(batch[:20])       # [3081 3081   12   12    6    6  195  195]
    # print(labels[:20])      # [[5234] [12] [6] [3081] [195] [12] [2] [6]]

    # data_index는 반복문에서 batch_size // num_skips 만큼 증가한다.
    # 최정적인 data_index는 마지막을 지난 위치를 가리키게 되니까, 정확한 계산을 위해 앞으로 돌려놓을 필요가 있다.
    # span에 context 윈도우 전체 크기가 있으니까, span만큼 뒤로 이동한다.
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index


# generate_batch 함수 테스트. 아래 코드는 뒤쪽에서 사용하지 않는다.
batch, labels, data_index = generate_batch(data, batch_size=8, num_skips=2, skip_window=1, data_index=0)
for i in range(8):
    print('{} {} -> {} {}'.format(batch[i],     ordered_words[batch[i]],
                                  labels[i, 0], ordered_words[labels[i, 0]]))
# [출력 결과]
# 3081 originated -> 12 as
# 3081 originated -> 5234 anarchism
# 12 as -> 6 a
# 12 as -> 3081 originated
# 6 a -> 195 term
# 6 a -> 12 as
# 195 term -> 2 of
# 195 term -> 6 a

In [ ]:
def parameter_initialization(vocabulary_size, projection_size):
    """
    Initialization method can be changed. 
    
    Argument:
    vocabulary_size -- the size of the vocabulary
    projection_size -- the dimension of hidden layer in NN
    
    Return:
    parameters -- a python dictionary containing weights W_in, b_in, W_out, b_out
    """
    
    np.random.seed(0)
    parameters = {}
    
    parameters['W_in'] = np.random.randn(vocabulary_size, projection_size) * 0.01 
    # parameters["b_in"] = np.zeros((projection_size, 1))
    
    parameters['W_out'] = np.random.randn(projection_size, vocabulary_size) * 0.01
    # parameters['b_out'] = np.zeros((vocabulary_size, 1))
    
    assert (parameters['W_in'].shape == (vocabulary_size, projection_size))
    assert (parameters['W_out'].shape == (projection_size, vocabulary_size))
    
        
    return parameters

In [ ]:
def feed_forward_NN(batch, labels, parameters):
    """
    Arguments:
    batch_input -- input of Neural Network
    layers -- a list containing the dimensions of each layer
    parameters -- python dictionary containing weights W_in, b_in, W_out, b_out
    
    Returns:
    cost -- the cost function of the NN
    caches -- used in backpropagation
    """
    W_in = parameters['W_in']  # size of vocabulary_size, projection_size
    hidden = W_in.T[:, batch]
    W_out = parameters['W_out']
    output = np.dot(W_out.T, hidden)
    # need to implement negative sampling
    # b_in = parameters['b_in']
    # b_out = parameters['b_out']
    
    assert(hidden.shape == (W_in.shape[1], len(batch)))
    assert(output.shape == (W_out.shape[1], len(batch)))
    
    
    return output

In [ ]:
def negative_sampling():
    

In [ ]:
def compute_cost_function(output, labels):
    """
    Arguments:
    output -- the prediction vector shape of (vocabulary_size, batch_size)
    label -- true label vector, shape of (batch_size, 1)
    
    Returns: 
    cost -- cost function shape of ()
    """
    cost = np.sum(output + labels[1]) # needs to be changed.
    
    np.squeeze(cost)
    assert (cost.shape == ())
    
    
    return cost 

In [ ]:
def backpropagation():
    """
    Arguments:
    
    Returns:
    
    """

In [ ]:
def update_parameters(parameters, grads, learning_Rate):
    """
    Updates the parameters based on adaGradientDescent with learning rate decay
    Arguments:
    parameters -- python dict containing parameters before the updates
    grads -- python dict containing the gradient
    Returns:
    parameters -- python dict containing updated parameters 
    """
    
    

In [ ]:
def train(vocabulary, projection_size, num_of_n_words, epoch, learning_rate):
    
    
    parameters = parameter_initialization(vocabulary_size, projection_size)
    
    int_voc, word_to_int, int_to_word, most_frequent_n_words = word_numbering(vocabulary)
    
    for i in range(epoch):
        # train
        
    
    
    
    
    return paramters

In [ ]:
vocabulary_size = 50000


def build_dataset(words, n_words):
    # count : [['UNK', -1], ('the', 1061396), ('of', 593677), ('and', 416629), ...]
    # 크기는 50,000개. UNK가 들어 있고, -1을 뺐으니까 처음에 전달된 크기 사용.
    # 빈도가 높은 5만개 추출.
    # count에 포함된 마지막 데이터는 ('hif', 9). 9번 나왔는데 드물다고 얘기할 수 있는지는 의문.
    unique = collections.Counter(words)             # 중복 단어 제거
    orders = unique.most_common(n_words - 1)        # 단어에 대한 빈도 계산. 갯수를 지정하지 않으면 전체 계산.
    count = [['UNK', -1]]
    count.extend(orders)

    # dictionary : (UNK, 0) (the, 1) (of, 2) (and, 3) (one, 4) (in, 5) (a, 6) (to, 7)
    # 내용을 보면 단어에 번호를 매겼다는 것을 알 수 있다.
    dictionary = {}
    for word, _ in count:
        dictionary[word] = len(dictionary)

    # 위의 코드는 결국 0부터 1씩 증가하는 인덱스를 len(dictionary)로 표현했기 때문에
    # enumerate 함수를 사용한 아래처럼 표현할 수 있다. len(dictionary)는 코드가 모호하다.
    # for i, (word, _) in enumerate(count):
    #     dictionary[word] = i

    # dictionary = {word: i for i, (word, _) in enumerate(count)}

    # 단어 전체에 대해 인덱스 매핑. data는 단어를 가리키는 인덱스 리스트가 된다.
    # 인덱스를 계산하기 위해 딕셔너리 대신 리스트를 사용할 수도 있고, 얼핏 보면 좋아보일 수도 있다.
    # 리스트를 사용하면 이진 검색을 적용해야
    data = []
    for word in words:
        if word in dictionary:          # word가 dictionary에 존재한다면
            index = dictionary[word]
        else:
            index = 0                   # UNK는 0번째에 위치
            count[0][1] += 1            # 갯수 : 418391
        data.append(index)

    # dictionary와 reversed_dictionary 내용
    # 일련번호로 된 key와 value가 의미 있을까? 리스트로 처리하면 되지 않을까?
    # (UNK, 0) (the, 1) (of, 2) (and, 3) (one, 4) (in, 5) (a, 6) (to, 7) (zero, 8) (nine, 9) (two, 10)
    # (0, UNK) (1, the) (2, of) (3, and) (4, one) (5, in) (6, a) (7, to) (8, zero) (9, nine) (10, two)
    # reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

    # reversed_dictionary는 0부터 시작하는 일련번호를 갖기 때문에 딕셔너리로 만들면 오히려 어렵고 불편하다.
    # 아래 코드는 리스트와 같다는 것을 증명하는 코드.
    # a = list(dictionary.values())
    # print(type(a), len(a))
    # print(a[0], a[-1])
    #
    # b = list(range(len(dictionary.values())))
    # print(b[0], b[-1])
    #
    # assert a == b

    # reversed_dictionary 대신 key로 구성된 리스트 반환.
    # [(0, UNK) (1, the) (2, of) (3, and) (4, one)]에서 인덱스를 제외하고 구성한 리스트.
    # 원본에서는 dictionary 변수를 반환하고 있는데, 사용하지 않기 때문에 삭제.
    return data, count, list(dictionary.keys())

# data : 단어에 대한 인덱스만으로 구성된 리스트
# count : 단어와 빈도 쌍으로 구성된 리스트. 중요한 변수이지만, 이번 코드에서는 사용 안함.
# ordered_words : 빈도에 따라 정렬된 단어 리스트
data, count, ordered_words = build_dataset(vocabulary, vocabulary_size)

# print('Most common words (+UNK)', count[:5])
# print('Sample data', data[:10], [ordered_words[i] for i in data[:10]], sep='\n')
# print('-'*50)
del vocabulary, count               # 사용하지 않는 변수 삭제

# [출력 결과]
# Most common words (+UNK) [['UNK', 418390], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
# Sample data
# [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156]
# ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']

